In [1]:
import os
import shutil
from PIL import Image
import numpy as np
import scipy as sp
from scipy.spatial import distance
import time
from tqdm import tqdm_notebook

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# Device configuration
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 
#NVIDIA GT 630M - 2.1 <= 3.0 !!! cuDNN error: CUDNN_STATUS_ARCH_MISMATCH
device = torch.device('cpu')
device

device(type='cpu')

In [23]:
dir_ = "data"
market_path_train = "/Market-1501-v15.09.15/bounding_box_train"
market_path_test = "/Market-1501-v15.09.15/bounding_box_test"
market_path_query = "/Market-1501-v15.09.15/query"
dist_market_path_train = "/trainClasses"
dist_market_path_test = "/testClasses"
dist_market_path_query = "/queryClasses"
dist_market_path_smallTest = "/smallTest"
dist_market_path_smallQuery = "/smallQuery"
#files_train = os.listdir(dir_ + market_path_train)
#files_test = os.listdir(dir_ + market_path_test)
#files_query = os.listdir(dir_ + market_path_query)

In [24]:
def load_train_dataset(path, batch_size=1, shuffle=False):
    data_path = dir_ + path
    train_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=transforms.Compose([
            #torchvision.transforms.RandomHorizontalFlip(),
            torchvision.transforms.ToTensor()
        ])
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=shuffle
    )
    return train_loader

def load_test_dataset(path, batch_size=1, shuffle=False):
    data_path = dir_ + path
    train_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=torchvision.transforms.ToTensor()
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=shuffle
    )
    return train_loader

In [85]:
def find_distance_one(object_with_features, test_features):
    list_dist = []
    for index in range(len(test_features)):
        dist_ = distance.cosine(object_with_features[1], test_features[index][1])
        list_dist.append((test_features[index][0], dist_, index))
    list_dist.sort(key=lambda tup: tup[1])
    result = (object_with_features[0], list_dist, object_with_features[2])
    return result

def find_distance_all(query_features, test_features):
    result = []
    for i in range(len(query_features)):
        distances_for_object = find_distance_one(object_with_features=query_features[i],
                                                 test_features=test_features)
        result.append(distances_for_object)
    return result

def find_distance_all_v(query_features, test_features):
    result = []
    for i in tqdm_notebook(range(len(query_features))):
        distances_for_object = find_distance_one(object_with_features=query_features[i],
                                                 test_features=test_features)
        result.append(distances_for_object)
    return result

def mAP(rangs, path_="data/testClasses/"):
    #{ { str_path, { {str_path_r_1, p_r_1}, {str_path_r_2, p_r_2}, .. {str_path_r_n, p_r_1} } }, ... }

    len_rangs = len(rangs) #751
    #print(len_rangs)
    sum_mAP = 0

    for rang in rangs:
        person = str(rang[0])
        len_person = len(os.listdir(path_ + str("{:0>4s}".format(person)) + '/'))
        sum_P = 0
        sum_true = 0
        i = 0
        for rang_r in rang[1]:
            i += 1
            if rang_r[0] == person:
                sum_true += 1
                sum_P += sum_true/i
            #print("Person: ", person, rang_r[0].split('_')[0], sum_true/i)
        #print("Person:", person, "len_person:", str(len_person), "AP =", sum_P/len_person)
        sum_mAP += sum_P/len_person
    return sum_mAP/len_rangs

In [86]:
def test_model(model, test_loader, query_loader, path_, v=False):
    test_features = []
    print("Waiting output from test_loader...")
    with torch.no_grad():
        for images, labels in test_loader:
            outputs2, outputs1 = model(images)
            test_features.append((str(labels.tolist()[0]), outputs1, images[0].numpy()))
    len(test_features)

    query_features = []
    print("Waiting output from query_loader...")
    with torch.no_grad():
        for images, labels in query_loader:
            outputs2, outputs1 = model(images)
            query_features.append((str(labels.tolist()[0]), outputs1, images[0].numpy()))
    len(query_features)
    
    print("Waiting mAP...")
    if v == False:
        rangs = find_distance_all(query_features=query_features, test_features=test_features)
    if v == True:
        rangs = find_distance_all_v(query_features=query_features, test_features=test_features)
    return mAP(rangs=rangs, path_=path_)

In [77]:
#inplace = 3 - кол-во входных каналов (RGB)

class CNN_ReID(nn.Module):
    def __init__(self, inplanes=3, planes1=8, planes2=16, planes3=32, stride=1):
        super(CNN_ReID, self).__init__()

        self.layer1 = nn.Sequential(nn.Conv2d(inplanes, planes1, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(planes1),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(nn.Conv2d(planes1, planes2, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(planes2),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(nn.Conv2d(planes2, planes3, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(planes3),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        #self.drop_out = nn.Dropout() 
        self.fc1 = nn.Linear(16 * 8 * planes3, 128) # 16*8*32 = 4096
        self.fc2 = nn.Linear(128, 751) # 16*8*32 = 4096
    def forward(self, x): 
        out = self.layer1(x) 
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        #out = self.drop_out(out)
        out1 = self.fc1(out)
        out2 = self.fc2(out1)
        return out2, out1

In [78]:
def trainModel(num_epochs=10, weight_decay=0.001, momentum=0.9, learning_rate=0.01, name="model_0.ckpt",
               train_loader=None, test_loader=None, query_loader=None):
    model = CNN_ReID().to(device)
    criterion = nn.CrossEntropyLoss() # CrossEntropyLoss() объединяет и SoftMax, и кросс-энтропийную функцию потерь
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=momentum)

    total_step = len(train_loader)
    loss_list = []
    acc_list = []

    best_mAP = -1
    
    print("wd =",weight_decay, "momentum =", momentum, "lr = ", learning_rate)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device) 
            labels = labels.to(device)

            # Прямой запуск
            outputs2, outputs1 = model(images)
            loss = criterion(outputs2, labels)
            loss_list.append(loss.item())

            # Обратное распространение и оптимизатор
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Отслеживание точности
            total = labels.size(0)
            _, predicted = torch.max(outputs2.data, 1)
            correct = (predicted == labels).sum().item()
            acc_list.append(correct / total)
            
            if (i + 1) % (total_step-1) == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.8f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                              (correct / total)))
                
        new_mAP = test_model(model=model, test_loader=test_loader, query_loader=query_loader,
                             path_="data/smallTest/")
        if (new_mAP >= best_mAP):
            torch.save(model.state_dict(), name)
            best_mAP = new_mAP
            bestEpoch = epoch
            print("bestModel was found! mAP: {:.4f}, epoch: {:d}".format(best_mAP, bestEpoch))
                
                
    print("LOSS =", loss.item(), "wd =", weight_decay, "momentum =",
          momentum, "lr =", learning_rate, "best_mAP =", best_mAP, "bestEpoch =", bestEpoch)

# Train

In [79]:
train_loader = load_train_dataset(dist_market_path_train, batch_size=1000, shuffle=True)

test_loader = load_test_dataset(dist_market_path_smallTest, batch_size=1, shuffle=False)
query_loader = load_test_dataset(dist_market_path_smallQuery, batch_size=1, shuffle=False)

without RandomHorizontalFlip()

In [80]:
start_time = time.time()
trainModel(num_epochs=10, weight_decay=0.001, momentum=0.9, learning_rate=0.05, name="model_Test_MAP_2.ckpt",
          train_loader=train_loader, test_loader=test_loader, query_loader=query_loader)
print("Time = ", time.time() - start_time)

wd = 0.001 momentum = 0.9 lr =  0.05
Epoch [1/10], Step [12/13], Loss: 6.2163, Accuracy: 0.03400000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.2169, epoch: 0
Epoch [2/10], Step [12/13], Loss: 5.0063, Accuracy: 0.13300000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.2882, epoch: 1
Epoch [3/10], Step [12/13], Loss: 3.5496, Accuracy: 0.33600000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.3257, epoch: 2
Epoch [4/10], Step [12/13], Loss: 2.4054, Accuracy: 0.48400000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.3623, epoch: 3
Epoch [5/10], Step [12/13], Loss: 1.4923, Accuracy: 0.65900000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.3940, epoch: 

In [81]:
start_time = time.time()
trainModel(num_epochs=20, weight_decay=0.001, momentum=0.9, learning_rate=0.01, name="model_Test_MAP_3.ckpt",
          train_loader=train_loader, test_loader=test_loader, query_loader=query_loader)
print("Time = ", time.time() - start_time)

wd = 0.001 momentum = 0.9 lr =  0.01
Epoch [1/20], Step [12/13], Loss: 6.5568, Accuracy: 0.00500000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.1089, epoch: 0
Epoch [2/20], Step [12/13], Loss: 6.3617, Accuracy: 0.01400000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.1390, epoch: 1
Epoch [3/20], Step [12/13], Loss: 6.1505, Accuracy: 0.04800000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.1581, epoch: 2
Epoch [4/20], Step [12/13], Loss: 5.8743, Accuracy: 0.06100000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.1755, epoch: 3
Epoch [5/20], Step [12/13], Loss: 5.5236, Accuracy: 0.09300000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.1847, epoch: 

In [82]:
start_time = time.time()
trainModel(num_epochs=30, weight_decay=0.001, momentum=0.9, learning_rate=0.005, name="model_Test_MAP_4.ckpt",
          train_loader=train_loader, test_loader=test_loader, query_loader=query_loader)
print("Time = ", time.time() - start_time)

wd = 0.001 momentum = 0.9 lr =  0.005
Epoch [1/30], Step [12/13], Loss: 6.5853, Accuracy: 0.00200000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.1452, epoch: 0
Epoch [2/30], Step [12/13], Loss: 6.4592, Accuracy: 0.02100000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.1684, epoch: 1
Epoch [3/30], Step [12/13], Loss: 6.3513, Accuracy: 0.02600000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.1944, epoch: 2
Epoch [4/30], Step [12/13], Loss: 6.1775, Accuracy: 0.03700000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.2162, epoch: 3
Epoch [5/30], Step [12/13], Loss: 6.0818, Accuracy: 0.04900000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.2368, epoch:

# Train with Self-Attention GAN

In [49]:
class Self_Attn(nn.Module):
    """ Self attention Layer"""
    def __init__(self,in_dim):
        super(Self_Attn,self).__init__()
        self.chanel_in = in_dim
        
        self.query_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.key_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.value_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim , kernel_size= 1)
        self.gamma = nn.Parameter(torch.zeros(1))

        self.softmax  = nn.Softmax(dim=-1) #
    def forward(self,x):
        """
            inputs :
                x : input feature maps( B X C X W X H)
            returns :
                out : self attention value + input feature 
                attention: B X N X N (N is Width*Height)
        """
        m_batchsize,C,width ,height = x.size()
        proj_query  = self.query_conv(x).view(m_batchsize,-1,width*height).permute(0,2,1) # B X CX(N)
        proj_key =  self.key_conv(x).view(m_batchsize,-1,width*height) # B X C x (*W*H)
        energy =  torch.bmm(proj_query,proj_key) # transpose check
        attention = self.softmax(energy) # BX (N) X (N) 
        proj_value = self.value_conv(x).view(m_batchsize,-1,width*height) # B X C X N

        out = torch.bmm(proj_value,attention.permute(0,2,1) )
        out = out.view(m_batchsize,C,width,height)
        
        out = self.gamma*out + x
        return out

In [54]:
#inplace = 3 - кол-во входных каналов (RGB)

class CNN_ReID_SAGAN(nn.Module):
    def __init__(self, inplanes=3, planes1=8, planes2=16, planes3=32, stride=1):
        super(CNN_ReID_SAGAN, self).__init__()

        self.layer1 = nn.Sequential(nn.Conv2d(inplanes, planes1, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(planes1),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(nn.Conv2d(planes1, planes2, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(planes2),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(nn.Conv2d(planes2, planes3, kernel_size=3, stride=1, padding=1),
                                    Self_Attn(planes3),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(planes3),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        #self.drop_out = nn.Dropout() 
        self.fc1 = nn.Linear(16 * 8 * planes3, 128) # 16*8*32 = 4096
        self.fc2 = nn.Linear(128, 751) # 16*8*32 = 4096
    def forward(self, x): 
        out = self.layer1(x) 
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        #out = self.drop_out(out)
        out1 = self.fc1(out)
        out2 = self.fc2(out1)
        return out2, out1

In [55]:
def trainModel_SAGAN(num_epochs=10, weight_decay=0.001, momentum=0.9, learning_rate=0.01, name="model_0.ckpt",
               train_loader=None, test_loader=None, query_loader=None):
    model = CNN_ReID_SAGAN().to(device)
    criterion = nn.CrossEntropyLoss() # CrossEntropyLoss() объединяет и SoftMax, и кросс-энтропийную функцию потерь
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=momentum)

    total_step = len(train_loader)
    loss_list = []
    acc_list = []

    best_mAP = -1
    
    print("wd =",weight_decay, "momentum =", momentum, "lr = ", learning_rate)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device) 
            labels = labels.to(device)

            # Прямой запуск
            outputs2, outputs1 = model(images)
            loss = criterion(outputs2, labels)
            loss_list.append(loss.item())

            # Обратное распространение и оптимизатор
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Отслеживание точности
            total = labels.size(0)
            _, predicted = torch.max(outputs2.data, 1)
            correct = (predicted == labels).sum().item()
            acc_list.append(correct / total)
            
            if (i + 1) % (total_step-1) == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.8f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                              (correct / total)))
                
        new_mAP = test_model(model=model, test_loader=test_loader, query_loader=query_loader,
                             path_="data/smallTest/")
        if (new_mAP >= best_mAP):
            torch.save(model.state_dict(), name)
            best_mAP = new_mAP
            bestEpoch = epoch
            print("bestModel was found! mAP: {:.4f}, epoch: {:d}".format(best_mAP, bestEpoch))
                
                
    print("LOSS =", loss.item(), "wd =", weight_decay, "momentum =",
          momentum, "lr =", learning_rate, "best_mAP =", best_mAP, "bestEpoch =", bestEpoch)

In [72]:
train_loader = load_train_dataset(dist_market_path_train, batch_size=500, shuffle=True)

test_loader = load_test_dataset(dist_market_path_smallTest, batch_size=1, shuffle=False)
query_loader = load_test_dataset(dist_market_path_smallQuery, batch_size=1, shuffle=False)

In [74]:
start_time = time.time()
trainModel(num_epochs=10, weight_decay=0.001, momentum=0.9, learning_rate=0.05, name="model_Test_SAGAN_MAP_2.ckpt",
          train_loader=train_loader, test_loader=test_loader, query_loader=query_loader)
print("Time = ", time.time() - start_time)

wd = 0.001 momentum = 0.9 lr =  0.05
Epoch [1/10], Step [25/26], Loss: 5.8638, Accuracy: 0.04600000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.2051, epoch: 0
Epoch [2/10], Step [25/26], Loss: 3.8071, Accuracy: 0.25400000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.3046, epoch: 1
Epoch [3/10], Step [25/26], Loss: 2.0473, Accuracy: 0.55800000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.3477, epoch: 2
Epoch [4/10], Step [25/26], Loss: 1.1170, Accuracy: 0.72800000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.3706, epoch: 3
Epoch [5/10], Step [25/26], Loss: 0.4242, Accuracy: 0.89400000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.3767, epoch: 

In [75]:
start_time = time.time()
trainModel(num_epochs=20, weight_decay=0.001, momentum=0.9, learning_rate=0.01, name="model_Test_SAGAN_MAP_3.ckpt",
          train_loader=train_loader, test_loader=test_loader, query_loader=query_loader)
print("Time = ", time.time() - start_time)

wd = 0.001 momentum = 0.9 lr =  0.01
Epoch [1/20], Step [25/26], Loss: 6.4061, Accuracy: 0.02200000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.2012, epoch: 0
Epoch [2/20], Step [25/26], Loss: 5.6999, Accuracy: 0.07000000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.2465, epoch: 1
Epoch [3/20], Step [25/26], Loss: 4.9327, Accuracy: 0.16400000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.2685, epoch: 2
Epoch [4/20], Step [25/26], Loss: 4.1648, Accuracy: 0.23800000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.3016, epoch: 3
Epoch [5/20], Step [25/26], Loss: 3.5450, Accuracy: 0.30800000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.3259, epoch: 

In [76]:
start_time = time.time()
trainModel(num_epochs=30, weight_decay=0.001, momentum=0.9, learning_rate=0.005, name="model_Test_SAGAN_MAP_4.ckpt",
          train_loader=train_loader, test_loader=test_loader, query_loader=query_loader)
print("Time = ", time.time() - start_time)

wd = 0.001 momentum = 0.9 lr =  0.005
Epoch [1/30], Step [25/26], Loss: 6.4419, Accuracy: 0.01800000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.1993, epoch: 0
Epoch [2/30], Step [25/26], Loss: 6.1663, Accuracy: 0.04400000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.2679, epoch: 1
Epoch [3/30], Step [25/26], Loss: 5.6911, Accuracy: 0.10200000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.3049, epoch: 2
Epoch [4/30], Step [25/26], Loss: 5.2847, Accuracy: 0.12400000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.3275, epoch: 3
Epoch [5/30], Step [25/26], Loss: 4.7594, Accuracy: 0.19800000%
Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...
bestModel was found! mAP: 0.3363, epoch:

# Test for models

### model_Test_SAGAN_MAP_2.ckpt

In [83]:
model = CNN_ReID_SAGAN()
model.load_state_dict(torch.load("model_Test_SAGAN_MAP_2.ckpt"))
model.eval()

CNN_ReID_SAGAN(
  (layer1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Self_Attn(
      (query_conv): Conv2d(32, 4, kernel_size=(1, 1), stride=(1, 1))
      (key_conv): Conv2d(32, 4, kernel_size=(1, 1), stride=(1, 1))
      (value_conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
      (softmax): Softmax()
    )
    (2): ReLU()
    (3): BatchNorm2d(32, eps=1

In [84]:
test_loader = load_test_dataset(dist_market_path_test, batch_size=1, shuffle=False)
query_loader = load_test_dataset(dist_market_path_query, batch_size=1, shuffle=False)

In [87]:
test_model(model=model, test_loader=test_loader, query_loader=query_loader, path_="data/testClasses/", v=True)

Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...


0.293543308106282

### model_Test_SAGAN_MAP_3.ckpt

In [88]:
model = CNN_ReID_SAGAN()
model.load_state_dict(torch.load("model_Test_SAGAN_MAP_3.ckpt"))
model.eval()

CNN_ReID_SAGAN(
  (layer1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Self_Attn(
      (query_conv): Conv2d(32, 4, kernel_size=(1, 1), stride=(1, 1))
      (key_conv): Conv2d(32, 4, kernel_size=(1, 1), stride=(1, 1))
      (value_conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
      (softmax): Softmax()
    )
    (2): ReLU()
    (3): BatchNorm2d(32, eps=1

In [89]:
test_model(model=model, test_loader=test_loader, query_loader=query_loader, path_="data/testClasses/", v=True)

Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...


0.27169232904932383

### model_Test_SAGAN_MAP_4.ckpt

In [90]:
model = CNN_ReID_SAGAN()
model.load_state_dict(torch.load("model_Test_SAGAN_MAP_4.ckpt"))
model.eval()

CNN_ReID_SAGAN(
  (layer1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Self_Attn(
      (query_conv): Conv2d(32, 4, kernel_size=(1, 1), stride=(1, 1))
      (key_conv): Conv2d(32, 4, kernel_size=(1, 1), stride=(1, 1))
      (value_conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
      (softmax): Softmax()
    )
    (2): ReLU()
    (3): BatchNorm2d(32, eps=1

In [91]:
test_model(model=model, test_loader=test_loader, query_loader=query_loader, path_="data/testClasses/", v=True)

Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...


0.2929243130455769

### model_Test_MAP_2.ckpt

In [92]:
model = CNN_ReID()
model.load_state_dict(torch.load("model_Test_MAP_2.ckpt"))
model.eval()

CNN_ReID(
  (layer1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=751, bias

In [93]:
test_model(model=model, test_loader=test_loader, query_loader=query_loader, path_="data/testClasses/", v=True)

Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...


0.30609234937164664

### model_Test_MAP_3.ckpt

In [94]:
model = CNN_ReID()
model.load_state_dict(torch.load("model_Test_MAP_3.ckpt"))
model.eval()

CNN_ReID(
  (layer1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=751, bias

In [95]:
test_model(model=model, test_loader=test_loader, query_loader=query_loader, path_="data/testClasses/", v=True)

Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...


0.19714548518819736

### model_Test_MAP_4.ckpt

In [96]:
model = CNN_ReID()
model.load_state_dict(torch.load("model_Test_MAP_4.ckpt"))
model.eval()

CNN_ReID(
  (layer1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=751, bias

In [97]:
test_model(model=model, test_loader=test_loader, query_loader=query_loader, path_="data/testClasses/", v=True)

Waiting output from test_loader...
Waiting output from query_loader...
Waiting mAP...


0.24769193554996904